In [3]:
import pandas as pd
import numpy as np
from statistics import NormalDist

# Retrieve the allocation df
df = pd.read_pickle("China Daily Signals.pickle")

# Get the tables containing the leverage
leverage_table_long = pd.read_excel("Leverage Tables - China.xlsx", sheet_name="Long Leverage")
leverage_table_short = pd.read_excel("Leverage Tables - China.xlsx", sheet_name="Short Leverage")

df = pd.DataFrame(data = df[["VXFXI", "Signal"]])

# Calculate the 9d rolling window mean for VXFXI
df["VXFXI_SMA_9"] = df["VXFXI"].rolling(9).mean()

# Calculate the 9d rolling window standard deviation for VXFXI
df["VXFXI_StdDev_9"] = df["VXFXI"].rolling(9).std()

# Calculate Prob(SMA < x) for SMA ~ N( 9d rolling mean, 9d rolling standard deviation )
df["VXFXI_CumProb"] = df.apply(lambda x: NormalDist(x["VXFXI_SMA_9"], x["VXFXI_StdDev_9"]).cdf(x["VXFXI"]), axis=1)


# function: lookup_leverage
# purpose: lookup and retrieve the leverage corresponding to a given weighted probability
# inputs:
#        cum_prob (double) - required cumulative probability for which leverage is required
#        signal (int)      - current signal (1 = long / 0 = short)
# returns:
#        leverage (double) - corresponding leverage
def lookup_leverage(cum_prob, signal):
    if(np.isnan(cum_prob)):
        return 0.0
    if(signal == 1):
        match = (leverage_table_long["Lower"] <= cum_prob) & (leverage_table_long["Upper"] > cum_prob)
        lev = leverage_table_long["Leverage"][match]
    else:
        match = (leverage_table_short["Lower"] <= cum_prob) & (leverage_table_short["Upper"] > cum_prob)
        lev = leverage_table_short["Leverage"][match]
    return lev.values[0]

# Get the leverage for each date and include in column
df["Leverage"] = df.apply(lambda x: lookup_leverage(x["VXFXI_CumProb"], x["Signal"]), axis=1)

# Save the leverage to an excel file
df.to_excel("China Daily Leverage.xlsx")

# Alco save the allocations to a leverage file
df.to_pickle("China Daily Leverage.pickle")

